In [1]:
# !pip install tensorflow
!pip install tensorflow[and-cuda]

In [2]:
import tensorflow as tf
import os

2024-09-27 20:24:33.321770: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-27 20:24:33.331538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 20:24:33.342782: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 20:24:33.345990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 20:24:33.354677: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

I0000 00:00:1727493877.088380    7394 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727493877.109863    7394 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727493877.109901    7394 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
image_dir = os.path.join("../data", "imagenet_224")

In [6]:
class_names = os.listdir(image_dir)
num_classes = len(class_names)
image_height = 224
image_width = 224
image_size = (image_height, image_width)
batch_size = 1

In [7]:
training_set = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    validation_split=0.3,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size)

validation_set = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=class_names,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size)


Found 14738 files belonging to 10 classes.
Using 10317 files for training.


I0000 00:00:1727493882.119119    7394 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727493882.119189    7394 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727493882.119200    7394 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727493882.308393    7394 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727493882.308439    7394 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-27

Found 14738 files belonging to 10 classes.
Using 2947 files for validation.


In [8]:
AUTOTUNE = tf.data.AUTOTUNE

training_set = training_set.cache().prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)     

In [9]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

In [11]:
input = Input(shape=(image_height, image_width, 3))

x = Conv2D(filters=32, kernel_size=3, activation="relu", padding="same")(input)
x = MaxPool2D(pool_size=2, strides=2)(x)
x = Conv2D(filters=32, kernel_size=3, activation="relu", padding="same")(x)
x = MaxPool2D(pool_size=2, strides=2)(x)
x = Dropout(0.4)(x)
x = Flatten()(x)
x = Dense(units=128, activation="relu")(x)

output = Dense(units=num_classes, activation="softmax")(x)

model = Model(input, output)

print(model.summary())



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_1 (Cast)                   │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 56, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,856,618 (49.04 MB)

 Trainable params: 12,856,618 (49.04 MB)

 Non-trainable params: 0 (0.00 B)

None


In [14]:
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[
                  CategoricalAccuracy(),
                  # TopKCategoricalAccuracy()
              ])

In [ ]:
epochs = 30
history = model.fit(training_set, validation_data=validation_set, epochs=epochs)

Epoch 1/30


I0000 00:00:1727493916.089628    7664 service.cc:146] XLA service 0x7f077c00b100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727493916.089664    7664 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-09-27 20:25:16.118254: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-27 20:25:16.253317: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


   33/10317 ━━━━━━━━━━━━━━━━━━━━ 49s 5ms/step - categorical_accuracy: 0.0538 - loss: 14.3169    

I0000 00:00:1727493919.380791    7664 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 1761/10317 ━━━━━━━━━━━━━━━━━━━━ 42s 5ms/step - categorical_accuracy: 0.1018 - loss: 14.3386

...

2024-09-27 20:25:30.106321: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1641] failed to alloc 145047808 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-09-27 20:25:30.106366: W external/local_xla/xla/stream_executor/integrations/device_host_allocator.h:61] could not allocate pinned host memory of size: 145047808


 1975/10317 ━━━━━━━━━━━━━━━━━━━━ 50s 6ms/step - categorical_accuracy: 0.1018 - loss: 14.3512

2024-09-27 20:25:31.311458: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1641] failed to alloc 2147483648 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-09-27 20:25:31.311512: W external/local_xla/xla/stream_executor/integrations/device_host_allocator.h:61] could not allocate pinned host memory of size: 2147483648
2024-09-27 20:25:31.322731: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1641] failed to alloc 1932735232 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-09-27 20:25:31.322763: W external/local_xla/xla/stream_executor/integrations/device_host_allocator.h:61] could not allocate pinned host memory of size: 1932735232
2024-09-27 20:25:31.366480: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1641] failed to alloc 1739461632 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-09-27 20:25:31.366514: W external/local_xla/xla/stream_executor/integrations/device_host_allocator.h:61] could not allocate pinn

 1982/10317 ━━━━━━━━━━━━━━━━━━━━ 52s 6ms/step - categorical_accuracy: 0.1018 - loss: 14.3515

2024-09-27 20:25:31.711118: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1641] failed to alloc 66365184 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-09-27 20:25:31.711134: W external/local_xla/xla/stream_executor/integrations/device_host_allocator.h:61] could not allocate pinned host memory of size: 66365184
2024-09-27 20:25:31.722149: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1641] failed to alloc 59728896 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-09-27 20:25:31.722163: W external/local_xla/xla/stream_executor/integrations/device_host_allocator.h:61] could not allocate pinned host memory of size: 59728896
2024-09-27 20:25:31.732623: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1641] failed to alloc 53756160 bytes on host: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-09-27 20:25:31.732637: W external/local_xla/xla/stream_executor/integrations/device_host_allocator.h:61] could not allocate pinned host me

In [ ]:
import pickle

with open(os.path.join("../data", "history_control.pkl"), "wb") as file:
    pickle.dump(history, file)

In [ ]:
import matplotlib.pyplot as plt

def plot_accuracy_curve(training_result, metric):
    val_metric = f"val_{metric}"
    train_perf = training_result.history[metric]
    validation_perf = training_result.history[val_metric]
    
    plt.plot(train_perf, label=metric)
    plt.plot(validation_perf, label=val_metric)
    
    max_val = max(validation_perf)
    max_val_epoch = validation_perf.index(max_val)
    
    plt.xlabel("Epoch")
    plt.ylabel(metric)
    plt.legend(loc="lower right")
    
# plot_accuracy_curve(history, "categorical_accuracy")
# plot_accuracy_curve(history, "top_k_categorical_accuracy")
plot_accuracy_curve(history, "loss")